# Midjourney Few Shot Example
This notebook shows how you can use llm prompting gen to create midjourney prompts. We will create a `ParsablePromptEngineeringGenerator` instance with the midjourney_prompt_gen template in order to create Text-to-Image prompts. The prompts should create images in the style of t-shirt designs, similar to the few-shot examples. 

## Setup
import required modules and create MidjourneyPromptGenerator instance

In [1]:
import os
import sys
from IPython.display import JSON
from typing import List
from pydantic import BaseModel, Field
from langchain.chat_models import ChatOpenAI
from llm_prompting_gen.generators import ParsablePromptEngineeringGenerator

In [2]:
if "notebooks" in os.getcwd():
    print("Change working directory to project root")
    os.chdir("..")

Change working directory to project root


Add your open ai key from https://platform.openai.com/account/api-keys

In [3]:
os.environ["OPENAI_API_KEY"] = ""

In [4]:
# Create pydantic in order to define output format 
class ImagePromptOutputModel(BaseModel):
    """LLM output format of image prompt generator"""
    few_shot_styles: List[str] = Field(description="Styles existing in the example prompts")
    few_shot_artists: List[str] = Field(description="Artists existing in the example prompts")
    image_prompts: List[str] = Field(description="List of text-to-image prompts")


In [5]:
llm = ChatOpenAI(temperature=0.7)
prompt_generator = ParsablePromptEngineeringGenerator.from_json("templates/midjourney_prompt_gen_shirt_design_cartoon_style.json", llm=llm, pydantic_cls=ImagePromptOutputModel)
# Change input text in order to match the pydantic target format
prompt_generator.prompt_elements.input = """
Complete the following tasks in the right order:
1. Try to extract the overarching styles or artists from the example prompts given to you by the instructor. Please only extract them if they appear in at least one example prompt.
2. Write five concise english prompts with the content "{text}". Your suggestions should include your found styles or artists of step 1 and use the same patterns as the example prompts.
    """

## Prompt Engineering Elements

In [6]:
JSON(prompt_generator.prompt_elements.dict())

/var/folders/5y/y83crpz5411g7gh8cnzf5_tc0000gn/T/ipykernel_24401/1543686729.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  JSON(prompt_generator.prompt_elements.dict())


<IPython.core.display.JSON object>

## Few shot prompt examples

In [8]:
prompt_examples = prompt_generator.prompt_elements.examples
for i, example in enumerate(prompt_examples[0:3]):
    print(f"Example {i}")
    print(example)

Example 0
A cartoonish frog, bright colors. T-shirt design graphic, vector, contour, white background, without any text
Example 1
realistic patriotic native American skull , cartoon character, vector art, process art, t shirt design, 8K, best quality, trending, perfect line work, perfect stroke, vector illustration, flat design, digital drawing, t shirt design --niji 5 --v 5
Example 2
A cartoonish motocyclist, bright colors. T - shirt design graphic, vector, contour, white background


## Prompt generation playground
Choose a simple text which gets transformed into high quality midjournes prompt.

In [9]:
text = """Sloth on tree"""
parsed_output: ImagePromptOutputModel = prompt_generator.generate(text)

In [10]:
JSON(parsed_output.dict())

/var/folders/5y/y83crpz5411g7gh8cnzf5_tc0000gn/T/ipykernel_24401/3978343034.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  JSON(parsed_output.dict())


<IPython.core.display.JSON object>

As we can see, we are now able to transform a simple text input into a comprehensive text to image prompt suitable for midjourney.